# ACMI Public API example matching internet biographies to Work creators

Written by our [ACMI Labs](https://labs.acmi.net.au) team as an example use of our pubilc collection API: [api.acmi.net.au](https://api.acmi.net.au)

Find the source on GitHub: https://github.com/ACMILabs/ACMI-API-x-Biographies


In [ ]:
# @title Licensed under the Mozilla Public License Version 2.0

# Copyright (c) 2021 ACMI Labs

# See the full license: https://github.com/ACMILabs/ACMI-API-x-Biographies/blob/main/LICENSE


# ACMI Public API

Let's search the ACMI Public API for all creators with a certain `name`. Enter the `name` you'd like to match below right, or if you leave it blank let's search for all creators who were `directors`.

In [ ]:
#@title ACMI API
#@markdown Add a firstname or lastname in the field below

import requests

# Search for ACMI creators with 'Simon' in their name
acmi_search_query = '' #@param {type:"string"}
acmi_search_query = acmi_search_query.lower()
creators = set()

if acmi_search_query:
    search_response = requests.get(
        'https://api.acmi.net.au/search/',
        params={
            'query': acmi_search_query,
            'field': 'creators_primary.name',
        },
    ).json()

    for result in search_response['results']:
        for creator in result.get('creators_primary'):
            if acmi_search_query in creator.get('name').lower():
                creators.add(creator.get('name'))

    print(
        f'Found {len(creators)} ACMI creators with {acmi_search_query} '
        f'in their name: {creators}'
    )
else:
    # Let's search for all directors instead
    search_response = requests.get(
        'https://api.acmi.net.au/search/',
        params={
            'query': 'director',
            'field': 'creators_primary.role',
        },
    ).json()

    for result in search_response['results']:
        for creator in result.get('creators_primary'):
            if 'director' in creator.get('role').lower():
                creators.add(creator.get('name'))

    print(f'Found {len(creators)} ACMI creators who are directors: {creators}')

# Wikipedia API

Next let's loop through the ACMI creators we found and try and match them to biographies from Wikipedia.

In [ ]:
# Search Wikipedia for information on each person
for creator in creators:
    wikipedia_search_response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': creator,
            'prop': 'info|extracts',
            'exintro': True,
            'explaintext': True,
            'inprop': 'url',
        },
    ).json()
    try:
        wikipedia_page = next(
            iter(wikipedia_search_response['query']['pages'].values()),
        )
        print(
            f'Wikipedia data for {creator}: {wikipedia_page["fullurl"]}\n'
            f'Title: {wikipedia_page["title"]}\n'
            f'Extract: {wikipedia_page["extract"]}'
        )
    except KeyError:
        print(f'Sorry, no data from Wikipedia for: {creator}')